In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing import image

# Define the image dimensions used during training.
# You MUST set these to the correct values for your model.
IMG_HEIGHT = 224
IMG_WIDTH = 224

def build_custom_cnn(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)):
    """
    Rebuilds the exact CNN model architecture used for training.
    This is necessary to load the weights correctly.
    """
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(2, activation='linear')  # Output 2 values: cos and sin
    ])
    return model

def predict_angle(img_path):
    """
    Loads a pre-trained model and uses it to predict the angle of a new image.

    Args:
        img_path (str): The file path to the image you want to predict.

    Returns:
        float: The predicted angle in degrees.
    """
    # Build the model architecture first
    model = build_custom_cnn(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    # Load the saved weights into the model
    try:
        model.load_weights("/cossin.h5")
    except Exception as e:
        print(f"Error loading weights: {e}")
        return None

    # Load the image and preprocess it
    try:
        img = image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
        img_array = image.img_to_array(img)

        # It is crucial to normalize the image in the same way it was normalized during training.
        # This example assumes pixel values were scaled to a [0, 1] range.
        img_array = img_array / 255.0

        # Expand dimensions to create a batch of 1
        img_array = np.expand_dims(img_array, axis=0)
    except Exception as e:
        print(f"Error loading or preprocessing image: {e}")
        return None

    # Make a prediction with the model
    prediction = model.predict(img_array)
    cos_pred, sin_pred = prediction[0]

    # Use arctan2 to get the angle in radians. It handles all four quadrants.
    angle_rad = np.arctan2(sin_pred, cos_pred)

    # Convert the angle to degrees
    angle_deg = np.rad2deg(angle_rad)

    # Normalize the angle to be within [0, 360) degrees
    angle_deg = (angle_deg + 360) % 360

    return angle_deg

# --- Example Usage ---
# Replace 'path/to/your/image.jpg' with a real image file path
test_image_path = "/6887a97e1c95c1200fdf0542.png"
predicted_angle = predict_angle(test_image_path)

if predicted_angle is not None:
    print(f"The predicted angle for the image is: {predicted_angle:.2f} degrees")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
The predicted angle for the image is: 0.37 degrees
